In [18]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from IPython import display
import itertools
from time import sleep

In [38]:
# CA
class CA:
    def __init__(self, x,y):
        self.grid = np.zeros((x,y))
        self.history = []
        self.x = x
        self.y = y
        
    def initialize_grid(self, per_cancer, num_alive):
        all_locations = ([tuple(x) for x in itertools.combinations_with_replacement(np.arange(10), 2)])
        num_cancer = math.floor(per_cancer*num_alive)
        assert (num_cancer + num_alive) < self.x*self.y
        cell_locations = random.sample(range(len(all_locations)),num_alive + num_cancer)
        cancer_locations = random.sample(cell_locations, num_cancer)
        self.grid[:][:] = 0 # empty
        for cell_location in cell_locations:
            self.grid[all_locations[cell_location][0], all_locations[cell_location][1]] = 1
        for cancer_location in cancer_locations:
            self.grid[all_locations[cancer_location][0], all_locations[cancer_location][1]] = 2
      
    def rule_set(self):
        return None
    
    def get_neighbours(self, grid_position, moore_neighborhood = True):
        neighbours = []
        i,j = grid_position
        if moore_neighborhood:
            neighbours.append(self.grid[i][j + self.])
                
    def print_grid(self):
            print(self.grid)

In [39]:
ca = CA(10,10)
per_cancer = 0.1 # percentage of alive cells
n_alive = 10
ca.initialize_grid(per_cancer, n_alive)
ca.print_grid()

[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 2. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
